In [ ]:
#export
from local.imports import *
from local.test import *
from local.core import *
from local.data.all import *
from local.notebook.showdoc import show_doc

In [ ]:
#default_exp tabular.core

# Tabular core

> Basic function to preprocess tabular data before assembling it in a `DataBunch`.

## TabularProc -

In [ ]:
#export
@docs
class TabularProc():
    "Base class to write a tabular processor for dataframes"
    order = 0
    def __init__(self, func=None):
        if func is not None: self.setup,self.__call__ = noop,func
    
    def setup(self, df, trn_idx=None): pass
    def __call__(self, df):  pass

    _docs = dict(setup="Use `df` to set its state, with just `trn_idx` if passed",
                 __call__="Process `df` with the state computed during setup`")

We use this class to preprocess tabular data. `cat_names` should contain the names of the categorical variables in your dataframe, `cont_names` the names of the continuous variables. If you don't need any state, you can initiliaze a `TabularProc` with a `func` to be applied on the dataframes. Otherwise you should subclass and implement `setup` and `__call__`.

In [ ]:
show_doc(TabularProc.setup)

<h4 id="<code>TabularProc.setup</code>" class="doc_header"><code>TabularProc.setup</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/40_tabular_core.ipynb#TabularProc--" class="source_link" style="float:right">[source]</a></h4>

> <code>TabularProc.setup</code>(**`df`**, **`trn_idx`**=*`None`*)

Use `df` to set its state, with just `trn_idx` if passed

In [ ]:
show_doc(TabularProc.__call__)

<h4 id="<code>TabularProc.__call__</code>" class="doc_header"><code>TabularProc.__call__</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/40_tabular_core.ipynb#TabularProc--" class="source_link" style="float:right">[source]</a></h4>

> <code>TabularProc.__call__</code>(**`df`**)

Process `df` with the state computed during setup`

In [ ]:
class Tabular():
    def __init__(self, df, cat_names=None, cont_names=None, cat_y=None):
        self.df,self.cat_names,self.cont_names,self.cat_y = df,L(cat_names),L(cont_names),cat_y
        
    def __getitem__(self, k): return self.df[k]
    def __setitem__(self, k, v): self.df[k] = v
        
    @property
    def loc(self): return self.df.loc
    @property
    def iloc(self): return self.df.iloc

In [ ]:
#export
from pandas.api.types import is_numeric_dtype, is_categorical_dtype

In [ ]:
#export
class Categorify(TabularProc):
    "Transform the categorical variables to that type."
    order = 1
    def setup(self, to, trn_idx=None):
        self.categories = {}
        for n in to.cat_names + to.cat_y: 
            col = to[n] if trn_idx is None else to.loc[trn_idx, n]
            self.categories[n] = pd.Categorical(col, ordered=True).categories
        
    def __call__(self, to):
        for n in to.cat_names + to.cat_y:
            to.loc[:,n] = pd.Categorical(to.loc[:,n], categories=self.categories[n], ordered=True)

In [ ]:
show_doc(Categorify, title_level=3)

<h3 id="<code>class</code> <code>Categorify</code>" class="doc_header"><code>class</code> <code>Categorify</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/40_tabular_core.ipynb#TabularProc--" class="source_link" style="float:right">[source]</a></h3>

> <code>Categorify</code>(**`func`**=*`None`*) :: [`TabularProc`](/tabular.core.html#TabularProc)

Transform the categorical variables to that type.

In [ ]:
cat = Categorify()
df = pd.DataFrame({'a':[0,1,2,0,2]})
to = Tabular(df, 'a')
cat.setup(to)
test_eq(list(cat.categories['a']), [0,1,2])
cat(to)
test_eq(df['a'].cat.codes.values, np.array([0,1,2,0,2]))
df1 = pd.DataFrame({'a':[1,0,3,-1,2]})
to1 = Tabular(df1, 'a')
cat(to1)
#Values that weren't in the training df are sent to -1 (na)
test_eq(df1['a'].cat.codes.values, np.array([1,0,-1,-1,2]))

In [ ]:
cat = Categorify()
df = pd.DataFrame({'a':[0,1,2,3,2]})
to = Tabular(df, 'a')
cat.setup(to, trn_idx=[0,1,2])
test_eq(list(cat.categories['a']), [0,1,2])
cat(to)
test_eq(df['a'].cat.codes.values, np.array([0,1,2,-1,2]))

In [ ]:
#export
class Normalize(TabularProc):
    "Normalize the continuous variables."
    order = 2
    def setup(self, to, trn_idx=None):
        self.means,self.stds = {},{}
        for n in to.cont_names:
            assert is_numeric_dtype(df[n]), (f"""Cannot normalize '{n}' column as it isn't numerical.
                Are you sure it doesn't belong in the categorical set of columns?""")
            col = (to[n] if trn_idx is None else to.loc[trn_idx,n])
            self.means[n],self.stds[n] = col.mean(),col.std(ddof=0)
    
    def __call__(self, to):
        for n in to.cont_names: to[n] = (to[n]-self.means[n]) / (1e-7 + self.stds[n])

In [ ]:
show_doc(Normalize, title_level=3)

<h3 id="<code>class</code> <code>Normalize</code>" class="doc_header"><code>class</code> <code>Normalize</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/40_tabular_core.ipynb#TabularProc--" class="source_link" style="float:right">[source]</a></h3>

> <code>Normalize</code>(**`func`**=*`None`*) :: [`TabularProc`](/tabular.core.html#TabularProc)

Normalize the continuous variables.

In [ ]:
norm = Normalize()
df = pd.DataFrame({'a':[0,1,2,3,4]})
to = Tabular(df, cont_names='a')
norm.setup(to)
x = np.array([0,1,2,3,4])
m,s = x.mean(),x.std()
test_eq(norm.means, {'a': m})
test_close(norm.stds['a'], s)
norm(to)
test_close(df['a'].values, (x-m)/s)
df1 = pd.DataFrame({'a':[5,6,7]})
to1 = Tabular(df1, cont_names='a')
norm(to1)
test_close(df1['a'].values, (np.array([5,6,7])-m)/s)

In [ ]:
norm = Normalize()
df = pd.DataFrame({'a':[0,1,2,3,4]})
to = Tabular(df, cont_names='a')
norm.setup(to, trn_idx=[0,1,2])
x = np.array([0,1,2])
m,s = x.mean(),x.std()
test_eq(norm.means, {'a': m})
test_close(norm.stds['a'], s)
norm(to)
test_close(df['a'].values, (np.array([0,1,2,3,4])-m)/s)

In [ ]:
#export
mk_class('FillStrategy', **{o:o for o in ['median', 'constant', 'most_common']})
FillStrategy.__doc__ = "Namespace containing the various filling strategies"

In [ ]:
#export
_all_ = ['FillStrategy']

In [ ]:
show_doc(FillStrategy, title_level=3)

<h3 id="<code>class</code> <code>FillStrategy</code>" class="doc_header"><code>class</code> <code>FillStrategy</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>FillStrategy</code>(**\*`args`**, **\*\*`kwargs`**)

Namespace containing the various filling strategies

In [ ]:
#export
class FillMissing(TabularProc):
    "Fill the missing values in continuous columns."
    def __init__(self, fill_strategy=FillStrategy.median, add_col=True, fill_val=0.):
        self.fill_strategy,self.add_col,self.fill_val = fill_strategy,add_col,fill_val
    
    def setup(self, to, trn_idx=None):
        self.na_dict = {}
        for n in to.cont_names:
            col = to[n] if trn_idx is None else to.loc[trn_idx,n]
            if pd.isnull(col).sum():
                if self.fill_strategy == FillStrategy.median: filler = col.median()
                elif self.fill_strategy == FillStrategy.constant: filler = self.fill_val
                else: filler = col.dropna().value_counts().idxmax()
                self.na_dict[n] = filler

    def __call__(self, to):
        for n in to.cont_names:
            if n in self.na_dict:
                if self.add_col: 
                    to[n+'_na'] = pd.isnull(to[n])
                    if n+'_na' not in to.cat_names: to.cat_names.append(n+'_na')
                to[n] = to[n].fillna(self.na_dict[n])
            elif pd.isnull(to[n]).sum() != 0:
                raise Exception(f"""There are nan values in field {n} but there were none in the training set given at setup. 
                Please fix those manually.""")

In [ ]:
show_doc(FillMissing, title_level=3)

<h3 id="<code>class</code> <code>FillMissing</code>" class="doc_header"><code>class</code> <code>FillMissing</code><a href="https://nbviewer.jupyter.org/github/fastai/fastai_docs/blob/master/dev/40_tabular_core.ipynb#TabularProc--" class="source_link" style="float:right">[source]</a></h3>

> <code>FillMissing</code>(**`fill_strategy`**=*`'median'`*, **`add_col`**=*`True`*, **`fill_val`**=*`0.0`*) :: [`TabularProc`](/tabular.core.html#TabularProc)

Fill the missing values in continuous columns.

In [ ]:
fill1,fill2,fill3 = (FillMissing(fill_strategy=s) 
                     for s in [FillStrategy.median, FillStrategy.constant, FillStrategy.most_common])
df = pd.DataFrame({'a':[0,1,np.nan,1,2,3,4]})
df1 = df.copy(); df2 = df.copy()
to,to1,to2 = Tabular(df, cont_names='a'),Tabular(df1, cont_names='a'),Tabular(df2, cont_names='a')
fill1.setup(to); fill2.setup(to1); fill3.setup(to2)
test_eq(fill1.na_dict, {'a': 1.5})
test_eq(fill2.na_dict, {'a': 0})
test_eq(fill3.na_dict, {'a': 1.0})

fill1(to); fill2(to1); fill3(to2)
for t in [to, to1, to2]: test_eq(t.cat_names, ['a_na'])

for df_,v in zip([to, to1, to2], [1.5, 0., 1.]):
    test_eq(df_['a'].values, np.array([0, 1, v, 1, 2, 3, 4]))
    test_eq(df_['a_na'].values, np.array([0, 0, 1, 0, 0, 0, 0]))
    
dfa = pd.DataFrame({'a':[np.nan,0,np.nan]})
dfa1 = dfa.copy(); dfa2 = dfa.copy()
to,to1,to2 = Tabular(dfa, cont_names='a'),Tabular(dfa1, cont_names='a'),Tabular(dfa2, cont_names='a')
fill1(to); fill2(to1); fill3(to2)
for df_,v in zip([to, to1, to2], [1.5, 0., 1.]):
    test_eq(df_['a'].values, np.array([v, 0, v]))
    test_eq(df_['a_na'].values, np.array([1, 0, 1]))

## TabularProcessor -

In [ ]:
#export
class TabularPreprocessor():
    "An object that will preprocess dataframes using `procs`"
    def __init__(self, procs, inplace=True):
        self.inplace = inplace
        self.procs = L(p() if isinstance(p, type) else TabularProc(func=p) for p in procs).sorted(key='order')
    
    def __call__(self, df, trn_idx=None):
        "Call each of `self.procs` on `df`, setup on `df[trn_idx]` if not None"
        to.df = to.df if self.inplace else to.df.copy()
        if trn_idx is None:
            for p in self.procs: p(to)
        else:
            self.procs,procs = [],self.procs
            for p in procs: 
                p.setup(to, trn_idx=trn_idx)
                p(to)
                if isinstance(p, Normalize): self.means,self.stds = p.means,p.stds
                self.procs.append(p)
            self.classes = {n:'#na#'+L(df[n].cat.categories, use_list=True) for n,c in df[to.cat_names + to.cat_y].items()}
            self.cat_names,self.cont_names,self.cat_y = to.cat_names,to.cont_names,to.cat_y

In [ ]:
class A(pd.DataFrame):
    ...

In [ ]:
df.c=['1']

/home/jhoward/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [ ]:
df.c

['1']

In [ ]:
df.b=[1]

In [ ]:
df = A(dict(a=[1]))

In [ ]:
df1 = df.reset_index()

In [ ]:
df1.b

AttributeError: 'DataFrame' object has no attribute 'b'

In [ ]:
df2 = A(df1)

In [ ]:
type(df2)

__main__.A

In [ ]:
df2

,index,a
0,0,1


In [ ]:
procs = [Normalize, Categorify, FillMissing, noop]
proc = TabularPreprocessor(procs, inplace=False)

#Test reordering and partialize
test_eq(L(proc.procs).mapped(type), [FillMissing, TabularProc, Categorify, Normalize])

df = pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,1,np.nan,1,2,3,4]})
to = Tabular(df, 'a', 'b')

#Test setup and apply on df_trn
proc(to, trn_idx=range_of(df))
test_eq(to['a'].cat.codes, [0,1,2,1,1,2,0])
test_eq(to['b_na'].cat.codes, [0,0,1,0,0,0,0])
x = np.array([0,1,1.5,1,2,3,4])
m,s = x.mean(),x.std()
test_close(to['b'].values, (x-m)/s)
test_eq(proc.classes, {'a': ['#na#',0,1,2], 'b_na': ['#na#',False,True]})

#Test apply on df_val
df = pd.DataFrame({'a':[2,1,3], 'b':[4,5,np.nan]})
to = Tabular(df, 'a', 'b')
proc(to)
test_eq(proc.classes, {'a': ['#na#',0,1,2], 'b_na': ['#na#',False,True]})
test_eq(to['a'].cat.codes, [2,1,-1])
test_eq(to['b_na'].cat.codes, [0,0,1])
x = np.array([4, 5, 1.5])
test_close(to['b'].values, (x-m)/s)

#Test apply on cat_y
procs = [Normalize, Categorify, FillMissing, noop]
proc = TabularPreprocessor(procs, inplace=False)

df = pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,1,np.nan,1,2,3,4], 'c': ['b','a','b','a','a','b','a']})
to = Tabular(df, 'a', 'b', cat_y='c')
proc(to, trn_idx=range_of(df))
test_eq(to.cat_names, ['a', 'b_na'])
test_eq(to['a'].cat.codes, [0,1,2,1,1,2,0])
test_eq(to['b_na'].cat.codes, [0,0,1,0,0,0,0])
test_eq(to['c'].cat.codes, [1,0,1,0,0,1,0])
x = np.array([0,1,1.5,1,2,3,4])
m,s = x.mean(),x.std()
test_close(to['b'].values, (x-m)/s)
test_eq(proc.classes, {'a': ['#na#',0,1,2], 'b_na': ['#na#',False,True], 'c': ['#na#','a','b']})

In [ ]:
#export
def process_df(df, splits, procs, cat_names=None, cont_names=None, cat_y=None, inplace=True):
    "Process `df` with `procs` and returns the processed dataframe and the `TabularProcessor` associated"
    to = Tabular(df, cat_names, cont_names, cat_y)
    proc = TabularPreprocessor(procs, inplace=inplace)
    proc(to, trn_idx=splits[0])
    return to,proc

In [ ]:
procs = [Normalize, Categorify, FillMissing, noop]
df = pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,1,np.nan,1,2,3,4]})
df = pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,1,np.nan,1,2,3,4], 'c': ['b','a','b','a','a','b','a']})
to,proc = process_df(df, [list(range(7))], procs, 'a', 'b')

KeyError: "['b_na'] not in index"

Pass the same `splits` as you will use for splitting the data, so that the setup is only done on the training set. `cat_names` are the names of the categorical variables, `cont_names` the continous ones, `cat_y` are the names of the dependent variables that are categories. If `inplace=True`, processing is applied inplace, otherwis it creates a copy of `df`.

In [ ]:
#export
class TabularLine(pd.Series):
    "A line of a dataframe that knows how to show itself"
    def show(self, ctx=None, **kwargs):
        if ctx is None: return self
        else: return ctx.append(self)

In [ ]:
#export
class TensorTabular(tuple):
    
    def get_ctxs(self, max_samples=10, **kwargs):
        n_samples = min(self[0].shape[0], max_samples)
        df = pd.DataFrame(index = range(n_samples))
        return [df.iloc[i] for i in range(n_samples)]
    
    def display(self, ctxs): display_df(pd.DataFrame(ctxs))

In [ ]:
#export
class ReadTabLine(ItemTransform):
    def __init__(self, proc): 
        self.proc = proc
        self.o2is = {n: defaultdict(int, {v:i for i,v in enumerate(proc.classes[n])}) for n in proc.cat_names}
    
    def encodes(self, row): 
        cats = [self.o2is[n][row[n]] for n in self.proc.cat_names]
        conts = [row[n] for n in self.proc.cont_names]
        return TensorTabular((tensor(cats).long(),tensor(conts).float()))
    
    def decodes(self, o) -> TabularLine:
        dic = {c: self.proc.classes[c][v] for v,c in zip(o[0], self.proc.cat_names)}
        ms = getattr(self.proc, 'means', {c:0 for c in self.proc.cont_names})
        ss = getattr(self.proc, 'stds',  {c:1 for c in self.proc.cont_names})
        dic.update({c: (v*ss[c] + ms[c]).item() for v,c in zip(o[1], self.proc.cont_names)})
        return pd.Series(dic)

In [ ]:
#export
class ReadTabTarget(ItemTransform):
    def __init__(self, proc): 
        self.proc = proc
        assert len(proc.cat_y) == 1
        self.o2i = defaultdict(int, {v:i for i,v in enumerate(proc.classes[proc.cat_y[0]])})
    
    def encodes(self, row): return self.o2i[row[self.proc.cat_y[0]]]-1
    def decodes(self, o) -> Category: return self.proc.classes[self.proc.cat_y[0]][o+1]

In [ ]:
tds = TfmdDS(to.df, tfms=[[ReadTabLine(proc)], ReadTabTarget(proc)], use_list=None)

In [ ]:
enc = tds[1]
test_eq(enc[0][0], tensor([2,1]))
test_close(enc[0][1], tensor([-0.628828]))
test_eq(enc[1], 0)

dec = tds.decode(enc)
assert isinstance(dec[0], TabularLine)
test_eq(dec[0], pd.Series({'a': 1, 'b_na': False, 'b': 1}))
test_eq(dec[1], 'a')

test_stdout(lambda: print(tds.show_at(1)), """a               1
b_na        False
b               1
category        a
dtype: object""")

## Integration example

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')
df_trn,df_tst = df.iloc[:10000],df.iloc[10000:]
df_trn.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [ ]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [Categorify, FillMissing, Normalize]

In [ ]:
splits = RandomSplitter()(range_of(df_trn))
to,proc = process_df(df_trn, splits, procs=procs, cat_names=cat_names, cont_names=cont_names, cat_y="salary", inplace=False)

/home/sgugger/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:947: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]


KeyError: 'b'

In [ ]:
dsrc = DataSource(to.df, filts=splits, tfms=[[ReadTabLine(proc)], [ReadTabTarget(proc)]])

NameError: name 'splits' is not defined

In [ ]:
dbch = dsrc.databunch(bs=64)
dbch.show_batch()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,category
0,Private,HS-grad,Never-married,Transport-moving,Other-relative,White,False,29.000000,167737.000000,9.0,<50k
1,Self-emp-not-inc,12th,Married-civ-spouse,Craft-repair,Husband,White,False,62.000000,134768.000000,8.0,>=50k
2,Private,Some-college,Widowed,Sales,Other-relative,White,False,65.000000,143175.000000,10.0,<50k
3,Local-gov,Bachelors,Never-married,Prof-specialty,Not-in-family,White,False,26.000000,166295.000000,13.0,<50k
4,Local-gov,10th,Married-civ-spouse,Transport-moving,Husband,White,False,58.000000,310085.000000,6.0,<50k
5,Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,False,47.000000,191978.000000,10.0,>=50k
6,Private,HS-grad,Married-civ-spouse,Transport-moving,Husband,White,False,48.000000,138069.000000,9.0,<50k
7,Private,Some-college,Never-married,Handlers-cleaners,Own-child,White,False,24.000000,224954.000000,10.0,<50k
8,Private,Some-college,Never-married,Handlers-cleaners,Not-in-family,White,False,21.000002,111467.007812,10.0,<50k
9,Private,Assoc-voc,Divorced,Tech-support,Own-child,White,False,37.000000,126011.007812,11.0,<50k


In [ ]:
proc(df_tst)[proc.cont_names+proc.cat_names].head()

,age,fnlwgt,education-num,workclass,education,marital-status,occupation,relationship,race,education-num_na
10000,0.470200,1.337218,1.167953,Private,Bachelors,Married-civ-spouse,Adm-clerical,Husband,Asian-Pac-Islander,False
10001,-0.925433,1.250466,-0.436653,Private,HS-grad,Married-civ-spouse,Transport-moving,Husband,Other,False
10002,1.057834,0.151212,-1.238957,Private,11th,Divorced,Other-service,Not-in-family,White,False
10003,0.543654,-0.280547,-0.436653,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,False
10004,0.764017,1.439434,0.365650,Self-emp-inc,Assoc-voc,Married-civ-spouse,Exec-managerial,Husband,White,False


## Export -

In [ ]:
#hide
from local.notebook.export import notebook2script
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_core.ipynb.
Converted 01a_dataloader.ipynb.
Converted 01a_script.ipynb.
Converted 02_transforms.ipynb.
Converted 03_pipeline.ipynb.
Converted 04_data_external.ipynb.
Converted 05_data_core.ipynb.
Converted 06_data_source.ipynb.
Converted 07_vision_core.ipynb.
Converted 08_pets_tutorial.ipynb.
Converted 09_vision_augment.ipynb.
Converted 11_layers.ipynb.
Converted 12_optimizer.ipynb.
Converted 13_learner.ipynb.
Converted 14_callback_schedule.ipynb.
Converted 15_callback_hook.ipynb.
Converted 16_callback_progress.ipynb.
Converted 17_callback_tracker.ipynb.
Converted 18_callback_fp16.ipynb.
Converted 19_callback_mixup.ipynb.
Converted 20_metrics.ipynb.
Converted 21_tutorial_imagenette.ipynb.
Converted 30_text_core.ipynb.
Converted 31_text_data.ipynb.
Converted 32_text_models_awdlstm.ipynb.
Converted 33_test_models_core.ipynb.
Converted 34_callback_rnn.ipynb.
Converted 35_tutorial_wikitext.ipynb.
Converted 36_text_models_qrnn.ipynb.
Converted 40_tabula